In [2]:
import cv2
import numpy as np
import imutils

dimensionFrameRecord = 20
minGoodPoint_1, maxFrame_1Add, minFrame_1Set =   5, 15, 4 #neutral
minGoodPoint_3, maxFrame_3Add, minFrame_3Set =   3, 8, 4 #happy
minGoodPoint_4, maxFrame_4Add, minFrame_4Set =   5, 10, 4 #sad

sift = cv2.SIFT_create()
sift3 = cv2.SIFT_create()
sift4 = cv2.SIFT_create()

image1C = cv2.imread("./image/Ref_neutral.jpg")
image1G = cv2.cvtColor(image1C, cv2.IMREAD_GRAYSCALE)  
kp1, des1 = sift.detectAndCompute(image1G, None)

image3C = cv2.imread("./image/Ref_happy.jpg")
image3G = cv2.cvtColor(image3C, cv2.IMREAD_GRAYSCALE)  
kp3, des3 = sift3.detectAndCompute(image3G, None)

image4C = cv2.imread("./image/Ref_sad.jpg")
image4G = cv2.cvtColor(image4C, cv2.IMREAD_GRAYSCALE)  
kp4, des4 = sift4.detectAndCompute(image4G, None)

cap = cv2.VideoCapture("./video/Video_source.mp4")

ret, frame = cap.read()
height, width = frame.shape[:2]
posTextObj1 = (30, int(0.05*height))    # 10%
posTextObj3 = (30, int(0.10*height))    # 20%
posTextObj4 = (30, int(0.15*height))    # 30%
posTextObjA = (30, int(0.20*height))    # 40%

runGoodArray1 = np.zeros(dimensionFrameRecord)
runGoodArray3 = np.zeros(dimensionFrameRecord)
runGoodArray4 = np.zeros(dimensionFrameRecord)

Obj1_Total, Obj1_Adder = 0, 0
Obj3_Total, Obj3_Adder = 0, 0
Obj4_Total, Obj4_Adder = 0, 0

while(cap.isOpened()):
    ret, image2C = cap.read() # อ่าน VDO
    if ret == True:
        for iShift in range(dimensionFrameRecord-1):
            runGoodArray1[iShift] = runGoodArray1[iShift+1]
            runGoodArray3[iShift] = runGoodArray3[iShift+1]
            runGoodArray4[iShift] = runGoodArray4[iShift+1]
            
        image2G = cv2.cvtColor(image2C, cv2.IMREAD_GRAYSCALE) 
        kp2, des2 = sift.detectAndCompute(image2G, None) # หา kp ใน VDO
        #------------------------------------Count 1 Neutral type-----------------------------------
        good = []
        if len(kp2) != 0 :
            match = cv2.BFMatcher()
            matches = match.knnMatch(des1, des2, k=2)
            for i_matche in range(len(matches)):
                try:
                    m, n = matches[i_matche]
                except (ValueError):
                    pass
                else:
                    if m.distance < 0.5 * n.distance :
                        good.append(m)
        
        if len(good) >= minGoodPoint_1:
            runGoodArray1[dimensionFrameRecord-1] = 1  
        else:
            runGoodArray1[dimensionFrameRecord-1] = 0 
        
        summFrame = runGoodArray1.sum(dtype=np.int32)
        if summFrame <= minFrame_1Set:
            Obj1_Adder = 1
        if summFrame >= maxFrame_1Add:
            Obj1_Total = Obj1_Total + Obj1_Adder
            Obj1_Adder = 0
        
        textShow = "Neutral Face : "+ str(Obj1_Total)     
        cv2.putText(image2C, textShow, posTextObj1, cv2.FONT_HERSHEY_PLAIN, 2, (168,18,164), 2)
        
        #------------------------------------Count 2: HAPPY type-----------------------------------
        good2 = []
        if len(kp2) != 0 :
            match2 = cv2.BFMatcher()
            matches2 = match2.knnMatch(des3, des2, k=2)
            for i_matche in range(len(matches2)):
                try:
                    m, n = matches2[i_matche]
                except (ValueError):
                    pass
                else:
                    if m.distance < 0.5 * n.distance :
                        good2.append(m)
        
        if len(good2) >= minGoodPoint_3:
            runGoodArray3[dimensionFrameRecord-1] = 1  
        else:
            runGoodArray3[dimensionFrameRecord-1] = 0 
        
        summFrame2 = runGoodArray3.sum(dtype=np.int32)
        if summFrame2 <= minFrame_3Set:
            Obj3_Adder = 1
        if summFrame2 >= maxFrame_3Add:
            Obj3_Total = Obj3_Total + Obj3_Adder
            Obj3_Adder = 0
 
        textShow = "Happy Face  : "+ str(Obj3_Total)  
        cv2.putText(image2C, textShow, posTextObj3, cv2.FONT_HERSHEY_PLAIN, 2, (37,69,251), 2)
        
        #------------------------------------Count 3 SAD type-----------------------------------
        good3 = []
        if len(kp2) != 0 :
            match3 = cv2.BFMatcher()
            matches3 = match3.knnMatch(des4, des2, k=2)
            for i_matche in range(len(matches3)):
                try:
                    m, n = matches3[i_matche]
                except (ValueError):
                    pass
                else:
                    if m.distance < 0.5 * n.distance :
                        good3.append(m)
        
        if len(good3) >= minGoodPoint_4:
            runGoodArray4[dimensionFrameRecord-1] = 1  
        else:
            runGoodArray4[dimensionFrameRecord-1] = 0 
        
        summFrame3 = runGoodArray4.sum(dtype=np.int32)
        if summFrame3 <= minFrame_4Set:
            Obj4_Adder = 1
        if summFrame3 >= maxFrame_4Add:
            Obj4_Total = Obj4_Total + Obj4_Adder
            Obj4_Adder = 0
            
        textShow = "Sad Face    : "+ str(Obj4_Total)  
        cv2.putText(image2C, textShow, posTextObj4, cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255), 2)
        
        #--------------------------------Count All---------------------------------------
            
        textShow = "Count All    : "+str(Obj1_Total+Obj3_Total+Obj4_Total)  
        cv2.putText(image2C, textShow, posTextObjA, cv2.FONT_HERSHEY_PLAIN, 2, (255,0,0), 2)
        
        #-----------------------------------------------------------------------
        cv2.imshow('Frame',image2C)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else: 
        break

cap.release()
cv2.destroyAllWindows()
print("Happy Face = ", Obj1_Total)
print("Neutral Face = ", Obj3_Total)
print("Sad Face = ", Obj4_Total)
print("Total = ",Obj1_Total+Obj3_Total+Obj4_Total)

Happy Face =  1
Neutral Face =  0
Sad Face =  0
Total =  1
